In [1]:
#import
import pandas as pd
import numpy as np
import os
from datetime import datetime 

In [2]:
df_exercises = pd.read_pickle('df_exercises.pickle')

In [3]:
df_exercises = df_exercises.drop(['cardioWorkload', 'perceivedWorkload' ],axis = 1)

In [4]:
df_exercises = df_exercises.dropna(how='any')

In [5]:
df_exercises = df_exercises[~(df_exercises == 0).any(axis=1)]

In [6]:
###################################################################################

In [7]:
from sklearn.cluster import KMeans

In [8]:
kmeans = KMeans(n_clusters=10)

In [9]:
kmeans.fit(df_exercises.drop(['splitName' ],axis = 1))

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [10]:
df_exercises = df_exercises.reset_index(drop=True)
df_exercises['labels'] = kmeans.labels_

In [11]:
drills = df_exercises['splitName'].unique()

In [12]:
drills

array(['Game-11v11 with se', 'Session', '11v1', 'Passing-drill dami',
       'Functional-trainin', '4vs4-plus 2 or 4vs', '4vs4-plus 2 and 4v',
       'Third-game', 'Second-game', 'Fourth-game', 'Preparation-to the',
       'Set-pieces', 'Warm-up', '2nd-half', '1st-half',
       'Individualised-training', 'Possession-10v10 conquer line',
       'Game-11v11', 'game', '1st.half', '2nd.half',
       'defensive-phase counter attack', 'Football-tennis', '11vs1',
       '10v11', 'Game-11vs11', 'First-half', 'Second-half',
       'Warm-up pre match', '5v7-plus 1 per 2', '5v5-plus 3 per 2 2',
       '5v5-plus 3 per 2 1', 'Int-run 15 15 1', 'Int-run 15 15 3',
       'int-run 15 15 2', 'Int-run 20 40 2', 'Offensive-phase 11v1',
       'Second-possession', 'First-possession', 'Possession-6 6vs6 ',
       'Defensive-phase co', 'Second-game 9vs9', 'First-game 9vs9',
       'Possession-game 6v', 'First-intermittent', 'Set-pieces defensive',
       'Offensive-combinat', '4vs6-last 20 m', 'Defensive-li

In [13]:
columns = ['splitName']
for cluster in range(len(kmeans.cluster_centers_)):
    columns.append('cat' + str(cluster + 1))

In [14]:
columns

['splitName',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9',
 'cat10']

In [15]:
df_normalized = pd.DataFrame()

for drill in drills:
    df_specific_drill = df_exercises.loc[df_exercises['splitName'].isin([drill])]
    df_value_counts = pd.DataFrame(df_specific_drill['labels'].value_counts(normalize=True))
    
    new_row = []
    
    for i in range(len(columns[1:])):
        try:
            new_row.append(df_value_counts.loc[i]['labels'])
        except:
            new_row.append(0)
    
    new_row.insert(0, drill)            
    new_row = pd.Series(new_row)

    df_normalized = df_normalized.append(new_row, ignore_index=True)

In [16]:
df_normalized.columns = columns

In [17]:
df_normalized['splitName'] = df_normalized['splitName'].astype(str).str.lower()

In [18]:
df_normalized = df_normalized.sort_values('splitName').reset_index(drop=True)

In [19]:
df_normalized

,splitName,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10
0,10v11,0.011429,0.168571,0.000000,0.248571,0.540000,0.000000,0.022857,0.000000,0.000000,0.008571
1,10v11-plus 10v1,0.000000,0.285714,0.000000,0.071429,0.642857,0.000000,0.000000,0.000000,0.000000,0.000000
2,10v11-plus-10v1,0.031250,0.187500,0.000000,0.250000,0.468750,0.000000,0.031250,0.000000,0.000000,0.031250
3,10vs10-conquer lin,0.000000,0.272727,0.000000,0.000000,0.727273,0.000000,0.000000,0.000000,0.000000,0.000000
4,11v0,0.000000,0.400000,0.000000,0.000000,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000
5,11v1,0.000000,0.500000,0.010870,0.010870,0.402174,0.000000,0.076087,0.000000,0.000000,0.000000
6,11v1-and set piece,0.000000,0.083333,0.000000,0.250000,0.583333,0.000000,0.000000,0.000000,0.000000,0.083333
7,11v1-and strength,0.000000,0.428571,0.000000,0.000000,0.571429,0.000000,0.000000,0.000000,0.000000,0.000000
8,11v1-plus sprint,0.000000,0.000000,0.111111,0.111111,0.166667,0.000000,0.555556,0.055556,0.000000,0.000000
9,11v10,0.000000,0.259259,0.000000,0.148148,0.592593,0.000000,0.000000,0.000000,0.000000,0.000000
